In [1]:
import os
import re
import json
import faiss
import warnings
import huggingface_hub

from openai import OpenAI

from langchain.schema import Document
from langchain_community.vectorstores.faiss import FAISS

from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain_upstage import UpstageEmbeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker


os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore", category=FutureWarning)

from dotenv import load_dotenv
load_dotenv("../keys.env")

upstage_api_key = os.getenv("UPSTAGE_API_KEY")
os.environ['UPSTAGE_API_KEY'] = upstage_api_key

openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai_api_key

hf_token = os.getenv("HF_TOKEN")
huggingface_hub.login(hf_token)

/home/pervinco/miniconda3/envs/ir-project/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/pervinco/.cache/huggingface/token
Login successful


In [2]:
client = OpenAI()
model = "gpt-4o"

# client = OpenAI(
#     api_key=upstage_api_key,
#     base_url="https://api.upstage.ai/v1/solar"
# )
# model = "solar-pro"

In [9]:
def clean_json_response(response):
    # 코드 블록(예: ```json, ```) 제거
    cleaned_response = re.sub(r'```(?:json)?', '', response).strip()
    
    return cleaned_response

def query_expansion(query, model:str, client:OpenAI):
    content = (
        """
        당신은 한국어 질의를 이해하고 개선하여 검색 시스템이 정확한 문서를 찾을 수 있도록 돕는 전문가입니다.

        **당신의 임무:**

        1. **질의 분석**: 사용자의 질의를 면밀히 분석하여 의도를 깊이 이해합니다. 이때 체인 오브 소트(chain-of-thought) 방식으로 내부적으로 추론하지만, 최종 답변에는 포함하지 않습니다.

        2. **의도 유지 및 확장**: 사용자의 원래 의도를 유지하면서, 검색 시스템이 관련 문서를 더 잘 찾을 수 있도록 질의를 명확하고 상세하게 확장합니다.

        3. **다의어 처리**: 다의어가 포함된 경우, 문맥에서 가장 적절한 의미를 선택하여 질의를 명확히 합니다.

        4. **분야 명확화**: 키워드가 여러 분야에서 사용될 수 있다면, 사용자가 의도한 분야나 가장 가능성이 높은 분야를 명확히 합니다.

        5. **모호성 해소**: 질의가 불명확하거나 모호하다면, 일반적인 해석을 추가하여 검색의 정확도를 높입니다.

        **지침:**

        - 최종 답변에는 **분석이나 추론 과정**을 포함하지 말고, **확장된 자연스러운 질의만을 제공합니다**.
        - 반환 형식은 반드시 **JSON 포맷**이며, 모든 문자열은 쌍따옴표로 감싸야 합니다.
        - 형식 예시: `{ "query": "확장된 자연스러운 질의" }`

        **예시:**

        예제 입력:
        ```
        Thomas Alva Edison은 누구인가요?
        ```
        확장된 질의:
        ```json
        { "query": "토머스 알바 에디슨(Thomas Alva Edison)은 어떤 인물인가요?" }
        ```

        예제 입력:
        ```
        온난 전선이 발생하면 이후 날씨는 어떻게 되나?
        ```
        확장된 질의:
        ```json
        { "query": "기상학에서 온난 전선이 발생한 후의 날씨 변화와 기상 현상은 어떻게 되나요?" }
        ```

        예제 입력:
        ```
        난관의 기능에 대해 알려줘.
        ```
        확장된 질의:
        ```json
        { "query": "인체 생식 기관 중 난관의 역할과 기능은 무엇인가요?" }
        ```
        """
    )

    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role" : "system", "content" : content},
            {"role" : "user", "content" : query}
            ],
    )
        
    response = completion.choices[0].message.content
    response = clean_json_response(response)
        
    try:
        json_response = json.loads(response)
    except json.JSONDecodeError:
        return {"error": "Invalid JSON response", "response": response}
        
    return json_response


In [10]:
def load_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]

def load_query(query_path, model, client, output_path):
    queries = load_jsonl(query_path)
    expanded_queries = []

    for query_data in queries:
        eval_id = query_data['eval_id']
        msg_history = query_data['msg']
        
        conversation_history = " ".join([msg['content'] for msg in msg_history if msg['role'] == 'user'])
        
        expanded_query_result = query_expansion(conversation_history, model, client)
        print(eval_id)
        print(conversation_history)
        print(expanded_query_result['query'], '\n')
        
        expanded_queries.append({
            "eval_id": eval_id,
            "original_conversation": conversation_history,
            "expanded_query": expanded_query_result.get('query', 'Expansion failed')
        })
    
    # Save expanded queries to a jsonl file
    with open(output_path, 'w', encoding='utf-8') as f:
        for query in expanded_queries:
            json.dump(query, f, ensure_ascii=False)
            f.write('\n')

In [11]:
load_query("../dataset/eval.jsonl", model, client, "../dataset/expanded_query_v2.jsonl")

78
나무의 분류에 대해 조사해 보기 위한 방법은?
식물학 및 생물학에서 나무의 분류 체계와 분류 방법을 조사하는 방법은 무엇인가요? 

213
각 나라에서의 공교육 지출 현황에 대해 알려줘.
세계 각 나라의 공교육 분야 예산 지출 현황과 각국 정부가 교육에 투자하는 비율은 어떻게 되나요? 

107
기억 상실증 걸리면 너무 무섭겠다. 어떤 원인 때문에 발생하는지 궁금해.
기억 상실증의 원인과 발생 메커니즘은 무엇인가요? 

81
통학 버스의 가치에 대해 말해줘.
통학 버스가 학생들에게 제공하는 장점과 가치는 무엇인가요? 교통 안전성, 비용 절감, 시간 효율성 측면에서 설명해주세요. 

280
Dmitri Ivanovsky가 누구야?
과학자 드미트리 이바노프스키(Dmitri Ivanovsky)는 어떤 인물이고, 그의 주요 업적은 무엇인가요? 

10
피임을 하기 위한 방법중 약으로 처리하는 방법은 쓸만한가?
피임을 위해 약물(피임약)을 사용하는 방법의 효과성과 장단점에 대해서 알려주세요. 

100
헬륨이 다른 원소들과 반응을 잘 안하는 이유는?
헬륨 원소가 화학적으로 다른 원소들과 반응을 잘 하지 않는 이유는 무엇인가요? 

279
문맹 비율이 사회 발전에 미치는 영향은?
문맹 비율이 사회 발전과 경제, 교육, 사회적 측면에 미치는 영향은 무엇인가요? 

42
이란 콘트라 사건이 뭐야 이 사건이 미국 정치에 미친 영향은?
이란 콘트라 사건의 개요와 이 사건이 미국 정치에 미친 영향은 무엇인가요? 

308
자기장이 얼마나 센지 표현하는 방식은?
물리학에서 자기장의 세기를 표현하는 방식에는 어떤 것들이 있나요? 

205
피를 맑게 하고 몸 속의 노폐물을 없애는 역할을 하는 기관은?
인체에서 피를 맑게 하고 몸 속의 노폐물을 제거하는 역할을 하는 기관은 무엇인가요? 

289
글리코겐의 분해는 인체에서 왜 필요한가?
인체에서 글리코겐의 분해가 필요한 이유와 그 과정에서 발생하는 생리적 역할은 무엇인가요? 

268
빗방울이 점점 커지게 되는 요인은?
기상학에서